In [158]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

In [5]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [160]:
lreg = LogisticRegression(random_state=0, solver='lbfgs')
lreg.fit(train_vectors, newsgroups_train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [161]:
pred = lreg.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')

0.9173457508731082

### Global weights

In [25]:
global_nbayes_params = nbayes.coef_[0]

In [60]:
selected_params = np.abs(global_nbayes_params).argsort()[-10:][::-1]
selected_params_values = global_nbayes_params[selected_params]

In [61]:
selected_feature_names = []
for sel_p_val in selected_params:
    selected_feature_names.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george

### Local Naive Bayes weights

In [42]:
local_gt = test_vectors[idx].nonzero()[1]

In [55]:
local_weights = np.abs(global_nbayes_params[local_gt]).argsort()[-10:][::-1]

selected_local_feature_names = []
for sel_p_val in local_weights:
    selected_local_feature_names.append(list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(sel_p_val)])  # Prints george

### LIME weights

In [31]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [169]:
idx = 83
explained_class = 1
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=10, labels=(explained_class, ))
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

/home/amir/anaconda3/envs/dl/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Document id: 83
Probability(christian) = 0.452
True class: atheism


In [170]:
selected_features_lime = []
for e in exp.as_list():
    selected_features_lime.append(vectorizer.vocabulary_[e[0]])

### Evaluation

In [153]:
import copy
idx = 83
explicand = copy.copy(test_vectors[idx].toarray())

In [185]:
def predict_replacement(explicand, x_train, selected_features, model, explained_class):
    #base_pred = model.predict(explicand)[explained_class]
    result = 0
    for j in range(len(selected_features_lime)):
        explicand_copy = copy.copy(explicand)
        explicand_f_val = explicand_copy[:, selected_features_lime[j]][0]
        feature_values = train_vectors[:, selected_features_lime[j]].toarray().flatten()
        bin_count, bin_edge = np.histogram(feature_values, bins='scott')

        for i in range(0, len(bin_edge) - 1):
            if explicand_f_val > bin_edge[i] and  explicand_f_val<bin_edge[i+1]:
                bin_idx = i
                bin_avg = np.mean([bin_edge[i], bin_edge[i+1]])

        prior = bin_count[bin_idx] / len(feature_values)
        explicand_copy[:, selected_features_lime[j]] = bin_avg
    
        new_pred = model.predict_proba(explicand_copy)[0][explained_class]
        
        result += new_pred * prior
    
    return result

In [196]:
base_pred = lreg.predict_proba(explicand)[0][explained_class]

In [197]:
new_pred_method_1 = predict_replacement(explicand, train_vectors, selected_features_lime, lreg, explained_class)

In [199]:
explicand_copy = copy.copy(explicand)

for s_f_l in selected_features_lime:
    explicand_copy[:, s_f_l] = np.mean(train_vectors[:, s_f_l]) 

new_pred_method_2 = lreg.predict_proba(explicand_copy)[0][explained_class]

In [203]:
(base_pred - new_pred_method_1), (base_pred - new_pred_method_2)

(0.4300836130356512, -0.15239361205791208)

In [201]:
bin_count, bin_edge = np.histogram(feature_values, bins='scott')

In [202]:
for i in range(0, len(bin_edge)):
    if explicand_f_val > bin_edge[i] and  explicand_f_val<bin_edge[i+1]:
        bin_idx = i
        bin_avg = np.mean([bin_edge[i], bin_edge[i+1]])

In [138]:
prior = bin_count[bin_idx] / len(feature_values)

In [139]:
prior

0.0018535681186283596

In [143]:
explicand[:, selected_features_lime[0]] = bin_avg

In [142]:
nbayes.predict(explicand)

array([0])

In [147]:
selected_features_lime[0]

4103

In [146]:
explicand.shape

(1, 23035)